In [31]:
import pandas as pd
import ssl
import urllib.request, urllib.parse, urllib.error
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import glob
import os.path


In [32]:
rochelle = pd.read_excel(r"C:\Users\justi\Downloads\GLANSIS Spp. Tracking.xlsx", sheet_name = "Listed")


In [33]:
#rochelle.columns
namelist= []
for name in rochelle['Scientific Name']:
    namelist.append(name)

print(namelist)

['Cabomba caroliniana', 'Myriophyllum spicatum', 'Phragmites australis', 'Nymphoides peltata', 'Potamogeton crispus', 'Trapa natans', 'Corbicula fluminea', 'Gambusia affinis', 'Pimephales vigilax', 'Crassula helmsii', 'Pistia stratiotes', 'Stratiotes aloides', 'Veronica beccabunga', 'Rhithropanopeus harrisi', 'Thermocyclops crassus', 'Cherax destructor', 'Hypophthalmichthys molitrix', 'Hypophthalmichthys nobilis', 'Lepomis auritus', 'Limnoperna fortunei', 'Alnus glutinosa', 'Apeltes quadracus', 'Arundo donax', 'Branchiura sowerbyi', 'Channa argus', 'Cipangopaludina japonica', 'Cyclops strenuus', 'Didymosphenia geminata', 'Discostella (Cyclotella) pseudostelligera', 'Leuciscus idus', 'Neoergasilus japonicus', 'Procambarus clarkii', 'Psammonobiotus communis', 'Psammonobiotus dziwnowi', 'Psammonobiotus linearis', 'Syngnathus abaster', 'Agrostis gigantea', 'Butomus umbellatus', 'Carassius carassius', 'Carex acutiformis', 'Carex disticha', 'Chenopodium glaucum (Oxybasis glauca)', 'Cirsium p

In [34]:
# Ignore SSL certificate errors --from Beautiful Soup SI 206 textbook
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [35]:
#accessing search page of the herbarium
atlas_url = "https://bellatlas.umn.edu/collections/harvestparams.php"
atlas_html = urllib.request.urlopen(atlas_url, context=ctx).read()
#herbarium_html


In [61]:
#setting up driver for Selenium
driver = webdriver.Firefox() #???Should I write a script for multiple Browsers?
driver.maximize_window()
scraped_dataframe = pd.DataFrame()#(index=[0])
scraped_dataframe.reset_index(drop=True)
for speciesname in namelist[1:2]:
    #print(speciesname)
    driver.get(atlas_url) # Goes to atlas
    namebox = driver.find_element(by = By.XPATH, value = '//*[@id="taxa"]')
    searchbutton = driver.find_element(by = By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/form/div[2]/div[1]/button') #selects the advanced search button based off the pulled tag from right clicking the inspected element
    time.sleep(.5)
    namebox.clear()
    namebox.send_keys(speciesname) #'Cabomba caroliniana') #Sends in a name from Rochelle's list
    searchbutton.click()
#if there are not results
#continue to next search

#if there are results
    windowbefore = driver.window_handles[0]
    #resultcount = driver.find_element(by = By.XPATH, value ='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[4]/div[3]').text
    downloadbutton = driver.find_element(by=By.XPATH, value='/html/body/table/tbody/tr/td/div[4]/div/div[2]/div/div[1]/form/button')
    downloadbutton.click()
    windowafter = driver.window_handles[1]
    driver.switch_to.window(windowafter)
    time.sleep(.5)
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[4]/td[2]/div/input[2]').click() #utf8 encoding
    driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/form/fieldset/table/tbody/tr[5]/td[2]/div/input').click() #uncompresses download

    downloadbutton=driver.find_element(by=By.NAME, value= "submitaction") #OVERWRITES THE PREVIOUS DOWNLOAD BUTTON
    downloadbutton.click()

    folder_path = r'C:\Users\justi\Downloads'
    file_type = r'\*CSV'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    downloadeddata = pd.read_csv(max_file, encoding='utf-8')
    masterdataframe = pd.DataFrame()
    if masterdataframe.empty:
        masterdataframe = pd.DataFrame(data=downloadeddata)
        #print('Empty')
    else:
        masterdataframe = masterdataframe.append(pd.DataFrame(data = downloadeddata))
    
    

In [62]:
masterdataframe

,id,institutionCode,collectionCode,ownerInstitutionCode,basisOfRecord,occurrenceID,catalogNumber,otherCatalogNumbers,kingdom,phylum,...,verbatimDepth,verbatimElevation,disposition,language,recordEnteredBy,modified,sourcePrimaryKey-dbpk,collId,recordId,references
0,6318,MIN,Plants,NaN,PreservedSpecimen,8aa59e90-b0b1-4085-ac65-7d396f385597,911585,1134888,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:8aa59e90-b0b1-4085-ac65-7d396f385597,https://bellatlas.umn.edu:443/collections/indi...
1,7662,MIN,Plants,NaN,PreservedSpecimen,c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,906278,1134887,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:c459d154-b6d7-4eb8-a1db-91d79c8ee8b6,https://bellatlas.umn.edu:443/collections/indi...
2,10966,MIN,Plants,NaN,PreservedSpecimen,a5d03247-7ffa-4f8e-af0b-be7e3210c806,906219,1134883,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:a5d03247-7ffa-4f8e-af0b-be7e3210c806,https://bellatlas.umn.edu:443/collections/indi...
3,16127,MIN,Plants,NaN,PreservedSpecimen,45527e00-3766-4b45-afff-93d99f5e90e1,477374,NaN,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:45527e00-3766-4b45-afff-93d99f5e90e1,https://bellatlas.umn.edu:443/collections/indi...
4,18640,MIN,Plants,NaN,PreservedSpecimen,903158b0-1dff-44a0-9785-01017e8b0744,427253,1134886,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,NaN,2018-02-27 10:36:53,NaN,1,urn:uuid:903158b0-1dff-44a0-9785-01017e8b0744,https://bellatlas.umn.edu:443/collections/indi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,2999527,MIN,Plants,NaN,PreservedSpecimen,8a67c67f-88c8-4dc6-8afd-6aaf29500cd6,965998,1356756,NaN,NaN,...,NaN,NaN,NaN,en,skelt096,2022-02-25 16:42:30,NaN,1,urn:uuid:8a67c67f-88c8-4dc6-8afd-6aaf29500cd6,https://bellatlas.umn.edu:443/collections/indi...
60,3046475,UMD,DUL,NaN,PreservedSpecimen,9d89253e-42ac-4c47-9b07-05a00923cac5,54538,DUL-V-41425,Plantae,Spermatophyta,...,NaN,NaN,NaN,en,gameier@d.umn.edu,2022-06-09 13:29:52,NaN,15,urn:uuid:9d89253e-42ac-4c47-9b07-05a00923cac5,https://bellatlas.umn.edu:443/collections/indi...
61,3046477,UMD,DUL,NaN,PreservedSpecimen,e37c5f23-26d1-447f-bbc0-f06d1a714fee,54536,DUL-V-41423,Plantae,Spermatophyta,...,NaN,NaN,NaN,en,gameier@d.umn.edu,2022-06-09 13:30:23,NaN,15,urn:uuid:e37c5f23-26d1-447f-bbc0-f06d1a714fee,https://bellatlas.umn.edu:443/collections/indi...
62,3062643,MIN,Plants,NaN,PreservedSpecimen,207b0432-8a5a-41a3-95dc-a7c4d9b9e355,969458,1371064,Plantae,Spermatophyta,...,NaN,NaN,NaN,NaN,savel047,2023-02-03 13:41:09,NaN,1,urn:uuid:207b0432-8a5a-41a3-95dc-a7c4d9b9e355,https://bellatlas.umn.edu:443/collections/indi...
